In [1]:
# If needed on your laptop:
!pip install -q "transformers>=4.41" accelerate torch pandas tqdm huggingface_hub
!pip install python-dotenv


In [3]:
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
if not HF_TOKEN:
    raise SystemExit("❌ Hugging Face token not found. Please set HF_TOKEN in .env file.")


In [8]:
import os, re, json, time, csv
from pathlib import Path
from collections import defaultdict, OrderedDict

import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 🔧 EDIT this dataset path
UCF_PATH = "/Users/dihan_ahmed/DRIVE_1/UTS/4th semester/ilab/UCFCrime_Train.json"

ART_DIR = Path("artifacts")
ART_DIR.mkdir(parents=True, exist_ok=True)


In [9]:
#Load Llama 3.2 1B Instruct model

MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"   # swap here if you try other models later

# Prefer Apple Metal (MPS) on Mac; else CUDA; else CPU
if torch.backends.mps.is_available():
    device_map = {"": "mps"}; torch_dtype = torch.float16
elif torch.cuda.is_available():
    device_map = "auto";      torch_dtype = torch.float16
else:
    device_map = None;        torch_dtype = torch.float32

# Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    torch_dtype=torch_dtype,
    device_map=device_map,
    # attn_implementation="eager",  # uncomment if you ever see MPS attention warnings
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
print("✅ Using:", MODEL_ID, "| device:", device_map)


Device set to use mps


✅ Using: meta-llama/Llama-3.2-1B-Instruct | device: {'': 'mps'}


In [10]:
#dataset loader
def load_ucf(path: str):
    data = json.loads(Path(path).read_text(encoding="utf-8"))
    for vid, blob in data.items():
        timestamps = blob.get("timestamps", [])
        sentences  = blob.get("sentences", [])
        for i, sent in enumerate(sentences):
            ts = timestamps[i] if i < len(timestamps) else [None, None]
            yield {
                "video_id": vid,
                "sent_idx": i,
                "start": ts[0],
                "end": ts[1],
                "sentence": sent
            }

print("Sample:", list(load_ucf(UCF_PATH))[:2])


Sample: [{'video_id': 'Abuse001_x264', 'sent_idx': 0, 'start': 0.0, 'end': 5.3, 'sentence': 'A woman with short hair, slightly fat, wearing a white top and black pants stood in front of the table, picked up a book from the table, and opened it to read'}, {'video_id': 'Abuse001_x264', 'sent_idx': 1, 'start': 7.0, 'end': 8.5, 'sentence': 'A man wearing a white shirt and black pants entered the house and walked towards the short-haired and fat woman in front who was reading a book.'}]


In [11]:
# promt schema
SCHEMA_INSTRUCTIONS = """Analyze the following annotation of a surveillance scene.
Identify all people and objects. For each person/object, extract the following features:
- attributes (clothes, clothes colour, looks, age, gender, posture, style, shoes),
- actions (what they are doing),
- interactions (who/what they interact with).

Return ONLY a JSON array. Ensure all fields are present, even if empty.
Keys: id, type ("person" or "object"), attributes[], actions[], interactions[].
Rules:
- If the text says "woman/lady/girl" include "female" in attributes; if "man/boy" include "male".
- Do NOT put scene objects (house, table, road, ground, car, etc.) into person attributes.
- Use "unknown" (in attributes) if gender is unclear.
"""

FEW_SHOT_INPUT = 'The woman wearing a red top holding a child walks in the direction of the woman in the couple.'
FEW_SHOT_OUTPUT = [
  {"id":"person_1","type":"person","attributes":["female","red top"],
   "actions":["walking","holding a child"],"interactions":["child"]}
]

def build_prompt(sentence: str) -> str:
    return (
        SCHEMA_INSTRUCTIONS
        + "\nExample input:\n" + FEW_SHOT_INPUT
        + "\nExample output:\n" + json.dumps(FEW_SHOT_OUTPUT, ensure_ascii=False)
        + "\nNow extract for this input:\n" + sentence.strip()
        + "\nOutput JSON array only:"
    )


In [12]:
#strict JSON parser
MAX_NEW_TOKENS = 280   # smaller → faster; increase if you see truncation
TEMPERATURE = 0.0
TOP_P = 1.0
REPETITION_PENALTY = 1.05
RETRY_ATTEMPTS = 2
RETRY_SLEEP = 0.25

def _basic_repair(text: str) -> str:
    """Strip code fences, keep the outermost [...], and fix trailing commas & smart quotes."""
    t = text.strip()
    t = re.sub(r"^```json|```$", "", t, flags=re.IGNORECASE|re.MULTILINE).strip()
    s, e = t.find("["), t.rfind("]")
    if s != -1 and e != -1:
        t = t[s:e+1]
    t = t.replace("“", '"').replace("”", '"').replace("’", "'")
    t = re.sub(r",\s*]", "]", t)
    t = re.sub(r",\s*}", "}", t)
    return t

def _parse_or_none(text: str):
    t = _basic_repair(text)
    try:
        return json.loads(t)
    except Exception:
        m = re.search(r"\[[\s\S]*\]", t)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception:
                pass
    return None

def _normalize(items):
    """Ensure all keys exist with sensible defaults."""
    norm = []
    for i, it in enumerate(items or []):
        norm.append({
            "id": it.get("id", f"item_{i}"),
            "type": it.get("type", "object"),
            "attributes": it.get("attributes", []) or [],
            "actions": it.get("actions", []) or [],
            "interactions": it.get("interactions", []) or [],
        })
    return norm

def call_llm(sentence: str):
    """Generate structured JSON for one sentence; retry if invalid JSON."""
    prompt = build_prompt(sentence)
    for attempt in range(1 + RETRY_ATTEMPTS):
        out = generator(
            prompt,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False if TEMPERATURE==0 else True,
            temperature=TEMPERATURE if TEMPERATURE>0 else None,
            top_p=TOP_P,
            repetition_penalty=REPETITION_PENALTY,
            pad_token_id=tokenizer.eos_token_id,
            return_full_text=False
        )[0]["generated_text"]

        # Some pipelines echo the prompt; trim if present
        if out.strip().startswith(prompt.strip()):
            out = out[len(prompt):].strip()

        data = _parse_or_none(out)
        if isinstance(data, list):
            return _normalize(data)

        # tighten instruction and retry
        if attempt < RETRY_ATTEMPTS:
            prompt = ("Your previous answer was not valid JSON.\n"
                      "Respond with a STRICT JSON array only (no prose):\n") + prompt
            time.sleep(RETRY_SLEEP)

    print("JSON parse failed; returning []. Raw snippet:\n", out[:400])
    return []


In [13]:
#cleaning and aggregation
OBJECT_WORDS = {
    "house","table","ground","road","street","sidewalk","car","van","truck","bus",
    "motorbike","bicycle","bag","umbrella","bench","chair","door","window","wall",
    "book","ball","sign","badge","license plate","number plate","store","entrance",
    "hallway","parking","lot","weather","rain","snow"
}

def _dedup_preserve_order(items):
    seen = OrderedDict()
    for x in items:
        x = (x or "").strip()
        if x and x not in seen:
            seen[x] = True
    return list(seen.keys())

def aggregate_fields_for_sentence(extracted_items):
    """Merge attributes/actions/interactions across entities (gender remains in attributes)."""
    attrs, acts, inters = [], [], []
    for it in (extracted_items or []):
        it_attrs = it.get("attributes", []) or []
        it_attrs = [a for a in it_attrs if a.lower().strip() not in OBJECT_WORDS]  # drop obvious objects
        attrs.extend(it_attrs)
        acts.extend(it.get("actions", []) or [])
        inters.extend(it.get("interactions", []) or [])

    attrs = _dedup_preserve_order(attrs)
    acts  = _dedup_preserve_order(acts)
    inters= _dedup_preserve_order(inters)

    return ";".join(attrs), ";".join(acts), ";".join(inters)


In [14]:
# Run extractor and save results
LIMIT_VIDEOS = 10  # increase later

# group rows by video
by_vid = defaultdict(list)
for r in load_ucf(UCF_PATH):
    by_vid[r["video_id"]].append(r)

video_ids = list(by_vid.keys())[:LIMIT_VIDEOS]
print("Selected videos:", video_ids, "\nModel used:", MODEL_ID)

extracted_rows = []
for vid in video_ids:
    for row in tqdm(by_vid[vid], desc=f"Extracting {vid}"):
        items = call_llm(row["sentence"])
        extracted_rows.append({**row, "extracted": items})

# save full raw json (audit/debug)
raw_json_path = ART_DIR / "extracted_llama.json"
raw_json_path.write_text(json.dumps(extracted_rows, ensure_ascii=False, indent=2), encoding="utf-8")
print("Wrote:", raw_json_path)

# save final CSV (gender is inside attributes)
final_csv_path = ART_DIR / "ucf_llm_featrures_2.csv"   # <- your requested filename
with final_csv_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["video","scene_idx","t_start","t_end","sentence","attributes","actions","interactions"])
    for r in extracted_rows:
        attributes, actions, interactions = aggregate_fields_for_sentence(r.get("extracted", []))
        writer.writerow([
            r.get("video_id",""),
            r.get("sent_idx",""),
            r.get("start",""),
            r.get("end",""),
            r.get("sentence",""),
            attributes,
            actions,
            interactions
        ])

print("Wrote:", final_csv_path)


Selected videos: ['Abuse001_x264', 'Abuse002_x264', 'Abuse003_x264', 'Abuse004_x264', 'Abuse005_x264', 'Abuse006_x264', 'Abuse007_x264', 'Abuse008_x264', 'Abuse009_x264', 'Abuse010_x264'] 
Model used: meta-llama/Llama-3.2-1B-Instruct


Extracting Abuse001_x264:  44%|████▍     | 4/9 [00:55<01:33, 18.79s/it]

JSON parse failed; returning []. Raw snippet:
  
[{"id": "person_2", "type": "person", "attributes": ["male", "white shirt", "black pants"], "actions": ["approached", "pulled out a piece of red cloth"], "interactions": ["woman"]}] 
[{"id": "person_3", "type": "person", "attributes": ["unknown", "short-haired", "fat woman"], "actions": ["not paying attention"], "interactions": ["woman"]}] 
[{"id": "object_4", "type": "object", "attributes": [],


Extracting Abuse002_x264:  59%|█████▉    | 13/22 [01:23<02:11, 14.65s/it]

JSON parse failed; returning []. Raw snippet:
  [{"id": "vehicle_1", "type": "vehicle", "attributes": ["silver", "van"], "actions": ["driving", "opening the door"], "interactions": ["woman", "couple", "child"]}] [{"id": "vehicle_2", "type": "vehicle", "attributes": ["unknown", "unknown"], "actions": ["driving"], "interactions": ["woman", "couple", "child"]}] [{"id": "vehicle_3", "type": "vehicle", "attributes": ["unknown", "unknown"], "actions


...

JSON parse failed; returning []. Raw snippet:
  
[{"id": "person_2", "type": "person", "attributes": ["unknown", "unknown"], "actions": ["standing", "holding a child"], "interactions": ["unknown"]}] 
[{"id": "person_3", "type": "person", "attributes": ["unknown", "unknown"], "actions": ["standing", "holding a child"], "interactions": ["unknown"]}] 
[{"id": "person_4", "type": "person", "attributes": ["unknown", "unknown"], "actions": ["standin


Extracting Abuse010_x264:  56%|█████▌    | 5/9 [00:38<00:40, 10.15s/it]

JSON parse failed; returning []. Raw snippet:
  [{"id": "person_2", "type": "person", "attributes": ["unknown", "curly hair"], "actions": ["ran", "raised her hand"], "interactions": ["puppy", "struggling puppy"]}] [{"id": "object_3", "type": "object", "attributes": [], "actions": ["ran", "struggled"], "interactions": ["puppy"]}]


Extracting Abuse010_x264: 100%|██████████| 9/9 [00:46<00:00,  5.20s/it]

Wrote: artifacts/extracted_llama.json
Wrote: artifacts/ucf_llm_featrures_2.csv


In [15]:
df = pd.read_csv(final_csv_path)
df.head(10)


,video,scene_idx,t_start,t_end,sentence,attributes,actions,interactions
0,Abuse001_x264,0,0.0,5.3,"A woman with short hair, slightly fat, wearing...",female;short hair;slightly fat,picking up a book;opening it to read,table
1,Abuse001_x264,1,7.0,8.5,A man wearing a white shirt and black pants en...,male;white shirt;black pants,entered the house;walked towards,short-haired and fat woman
2,Abuse001_x264,2,7.2,8.5,A man wearing a black shirt and black pants en...,male;black shirt;black pants,entered the house;walked towards,short-haired and fat woman
3,Abuse001_x264,3,8.2,8.9,A man wearing a white shirt and black pants ap...,NaN,NaN,NaN
4,Abuse001_x264,4,8.9,11.2,A man in black clothes approached a short-hair...,male;black clothes,punched the woman in the head,woman
5,Abuse001_x264,5,8.9,11.2,"The woman fell to the ground in pain, and the ...",wooden;red,fell;knocked,table;book;woman
6,Abuse001_x264,6,11.3,13.3,A woman with short hair and a fat figure weari...,female;short hair;fat figure,fell to the ground;raised her right hand to to...,table leg
7,Abuse001_x264,7,15.2,18.9,"A woman with short hair, slightly fat, wearing...",unknown;short hair;slightly fat;white top;blac...,fell to the ground,house
8,Abuse001_x264,8,19.7,25.4,"A woman with short hair, slightly fat, wearing...",unknown;short hair;slightly fat,kneel down;stand up,ground
9,Abuse002_x264,0,0.0,2.3,"At an intersection with smooth traffic, the gr...",smooth;traffic;green light;vehicles on the opp...,none,none
